# Addition using QFT

## 1. Introduction <a id='introduction'></a>

The quantum Fourier transform (QFT) is the quantum implementation of the discrete Fourier transform over the amplitudes of a wavefunction. It is part of many quantum algorithms, like Shor's factoring algorithm and quantum phase estimation.

The quantum Fourier transform acts on a quantum state $\vert X\rangle = \sum_{j=0}^{N-1} x_j \vert j \rangle$ and maps it to the quantum state $\vert Y\rangle = \sum_{k=0}^{N-1} y_k \vert k \rangle$ according to the formula



\begin{equation}
y_k = \frac{1}{\sqrt{N}}\sum_{j=0}^{N-1}x_j\omega_N^{jk}
\end{equation}



with $\omega_N^{jk} = e^{2\pi i \frac{jk}{N}}$. Note that only the amplitudes of the state were affected by this transformation.

This can also be expressed as the map:


$$\vert j \rangle \mapsto \frac{1}{\sqrt{N}}\sum_{k=0}^{N-1}\omega_N^{jk} \vert k \rangle$$



Or the unitary matrix:


$$ U_{QFT} = \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} \sum_{k=0}^{N-1} \omega_N^{jk} \vert k \rangle \langle j \vert$$



## 2. Intuition <a id="intuition"></a>

The quantum Fourier transform (QFT) transforms between two bases, the computational (Z) basis, and the Fourier basis. The H-gate is the single-qubit QFT, and it transforms between the Z-basis states $|0\rangle$ and $|1\rangle$ to the X-basis states $|{+}\rangle$ and $|{-}\rangle$. In the same way, all multi-qubit states in the computational basis have corresponding states in the Fourier basis. The QFT is simply the function that transforms between these bases.

$$
|\text{State in Computational Basis}\rangle \quad \xrightarrow[]{\text{QFT}} \quad |\text{State in Fourier Basis}\rangle
$$

$$
\text{QFT}|x\rangle = |\widetilde{x}\rangle
$$

(States in the Fourier basis are often denoted using the tilde (~)).

All the states of the basis will be represented via qubits in the XY-plane of the Bloch sphere, each rotated by a certain amount. Suppose we are working with $n$ qubits and we want to represent the number $m$ in the Fourier basis. Then the j-th qubit will have the phase:
$$
\omega_j = \frac{2m\pi}{2^{j}}
$$

This can be easily seen in practice by expanding the action of the QFT for an arbitrary $N=2^n$ ($n$ qubits), acting on the state $\vert x \rangle = \vert x_1\ldots x_n \rangle$ where $x_1$ is the most significant bit:
$$
\begin{aligned}
QFT_N\vert x \rangle & = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1}\omega_N^{xy} \vert y \rangle
\\
& = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} e^{2 \pi i xy / 2^n} \vert y \rangle ~\text{since}\: \omega_N^{xy} = e^{2\pi i \frac{xy}{N}} \:\text{and}\: N = 2^n
\\
& = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} e^{2 \pi i \left(\sum_{k=1}^n y_k/2^k\right) x} \vert y_1 \ldots y_n \rangle \:\text{rewriting in fractional binary notation}\: y = y_1\ldots y_n, y/2^n = \sum_{k=1}^n y_k/2^k
\\
& = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} \prod_{k=1}^n e^{2 \pi i x y_k/2^k } \vert y_1 \ldots y_n \rangle \:\text{after expanding the exponential of a sum to a product of exponentials}
\\
& = \frac{1}{\sqrt{N}} \bigotimes_{k=1}^n  \left(\vert0\rangle + e^{2 \pi i x /2^k } \vert1\rangle \right) \:\text{after rearranging the sum and products, and expanding}
\sum_{y=0}^{N-1} = \sum_{y_1=0}^{1}\sum_{y_2=0}^{1}\ldots\sum_{y_n=0}^{1}
\\
& = \frac{1}{\sqrt{N}}
\left(\vert0\rangle + e^{\frac{2\pi i}{2}x} \vert1\rangle\right)
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^2}x} \vert1\rangle\right)
\otimes  
\ldots
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^{n-1}}x} \vert1\rangle\right)
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^n}x} \vert1\rangle\right)
\end{aligned}
$$

## 3. Performing addition with QFT <a id="addition1"></a>


### 3.1. One register <a id="addition1"></a>

The fact that the states encoding the numbers are now in phase gives us great flexibility in carrying out our arithmetic operations. To see this in practice, let’s look at the situation in which we want to create an operator `Sum(k)` such that:

$$\text{Sum}(k) |m\rangle = |m + k\rangle$$

The procedure to implement this unitary operation is the following:

1. We convert the state from the computational basis into the Fourier basis by applying the QFT to the $|m\rangle$ state via the QFT operator:

$$\text{QFT} |m\rangle = \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} \omega_N^{mj} |j\rangle$$

2. We rotate the j-th qubit by the angle $\frac{2k\pi}{2^j}$ using the $RZ$ gate, which leads to the new phases $\frac{2(m+k)\pi}{2^j}$.

3. We apply the inverse QFT and obtain $m+k$ in the computational basis.


Let us now try it. Finish the missing code by implementing step 2 with the function add_k_fourier.

In [1]:
!pip install qiskit
!pip install qiskit-aer
!pip install numpy

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer, AerSimulator
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_histogram
import numpy as np

# we use the little endian notation for bit ordering

n_wires = 3

def add_k_fourier(circuit, k, wires):
    # your code here
    for j in range(len(wires)):
        circuit.rz(k * np.pi / (2**(len(wires) - 1 - j)), wires[j])

def basis_embedding(circuit, m, wires):
    binary_m = bin(m)[2:]
    for i, bit in enumerate(binary_m):
        if bit == '1':
            circuit.x(len(binary_m) - i - 1)

def sum(m, k):

    qc = QuantumCircuit(n_wires)

    basis_embedding(qc, m, n_wires)

    qc.append(QFT(n_wires), range(n_wires))

    add_k_fourier(qc, k, range(n_wires))

    qc.append(QFT(n_wires, inverse=True), range(n_wires))

    qc.measure_all()
    print(qc.draw())

    sim = Aer.get_backend('aer_simulator')
    qc = transpile(qc, AerSimulator())
    result = sim.run(qc).result()
    counts = result.get_counts()

    return counts


print(f"The result of the sum is {sum(3,4)}")


        ┌───┐┌──────┐┌───────┐ ┌───────┐ ░ ┌─┐      
   q_0: ┤ X ├┤0     ├┤ Rz(π) ├─┤0      ├─░─┤M├──────
        ├───┤│      │├───────┴┐│       │ ░ └╥┘┌─┐   
   q_1: ┤ X ├┤1 QFT ├┤ Rz(2π) ├┤1 IQFT ├─░──╫─┤M├───
        └───┘│      │├────────┤│       │ ░  ║ └╥┘┌─┐
   q_2: ─────┤2     ├┤ Rz(4π) ├┤2      ├─░──╫──╫─┤M├
             └──────┘└────────┘└───────┘ ░  ║  ║ └╥┘
meas: 3/════════════════════════════════════╩══╩══╩═
                                            0  1  2 
The result of the sum is {'111': 1024}


/var/folders/0y/p40nt1td47xgs3zh4yrj4qwc0000gn/T/ipykernel_24294/1721904404.py:28: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_wires), range(n_wires))
/var/folders/0y/p40nt1td47xgs3zh4yrj4qwc0000gn/T/ipykernel_24294/1721904404.py:32: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_wires, inverse=True), range(n_wires))


By summing 3 and 4, you should obtain 111, which represents the correct answer (7) in binary. Now test the circuit with 3 and 6:

In [3]:
print(f"The result of the sum is {sum(3,6)}")

        ┌───┐┌──────┐┌──────────┐┌───────┐ ░ ┌─┐      
   q_0: ┤ X ├┤0     ├┤ Rz(3π/2) ├┤0      ├─░─┤M├──────
        ├───┤│      │└┬────────┬┘│       │ ░ └╥┘┌─┐   
   q_1: ┤ X ├┤1 QFT ├─┤ Rz(3π) ├─┤1 IQFT ├─░──╫─┤M├───
        └───┘│      │ ├────────┤ │       │ ░  ║ └╥┘┌─┐
   q_2: ─────┤2     ├─┤ Rz(6π) ├─┤2      ├─░──╫──╫─┤M├
             └──────┘ └────────┘ └───────┘ ░  ║  ║ └╥┘
meas: 3/══════════════════════════════════════╩══╩══╩═
                                              0  1  2 
The result of the sum is {'001': 1024}


/var/folders/0y/p40nt1td47xgs3zh4yrj4qwc0000gn/T/ipykernel_24294/1721904404.py:28: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_wires), range(n_wires))
/var/folders/0y/p40nt1td47xgs3zh4yrj4qwc0000gn/T/ipykernel_24294/1721904404.py:32: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_wires, inverse=True), range(n_wires))


What is the output and how does it relate to the number of qubits in our circuit?
For n qubits, you should obtain the number m + k (mod $2^n$).
Generalize the sum function so that the addition of any two numbers outputs the correct result.

In [4]:
## Adapt sum(m, k) to work with any m and k

def sum_any(m, k):

    n_wires = int(np.floor(np.log2(m+k))+1)

    qc = QuantumCircuit(n_wires)

    basis_embedding(qc, m, n_wires)

    qc.append(QFT(n_wires), range(n_wires))

    add_k_fourier(qc, k, range(n_wires))

    qc.append(QFT(n_wires, inverse=True), range(n_wires))

    qc.measure_all()
    print(qc.draw())

    sim = Aer.get_backend('aer_simulator')
    qc = transpile(qc, AerSimulator())
    result = sim.run(qc).result()
    counts = result.get_counts()

    return counts

sum_any(3,6)


        ┌───┐┌──────┐┌──────────┐┌───────┐ ░ ┌─┐         
   q_0: ┤ X ├┤0     ├┤ Rz(3π/4) ├┤0      ├─░─┤M├─────────
        ├───┤│      │├──────────┤│       │ ░ └╥┘┌─┐      
   q_1: ┤ X ├┤1     ├┤ Rz(3π/2) ├┤1      ├─░──╫─┤M├──────
        └───┘│  QFT │└┬────────┬┘│  IQFT │ ░  ║ └╥┘┌─┐   
   q_2: ─────┤2     ├─┤ Rz(3π) ├─┤2      ├─░──╫──╫─┤M├───
             │      │ ├────────┤ │       │ ░  ║  ║ └╥┘┌─┐
   q_3: ─────┤3     ├─┤ Rz(6π) ├─┤3      ├─░──╫──╫──╫─┤M├
             └──────┘ └────────┘ └───────┘ ░  ║  ║  ║ └╥┘
meas: 4/══════════════════════════════════════╩══╩══╩══╩═
                                              0  1  2  3 


/var/folders/0y/p40nt1td47xgs3zh4yrj4qwc0000gn/T/ipykernel_24294/4060017202.py:11: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_wires), range(n_wires))
/var/folders/0y/p40nt1td47xgs3zh4yrj4qwc0000gn/T/ipykernel_24294/4060017202.py:15: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_wires, inverse=True), range(n_wires))


{'1001': 1024}

### 3.1. Two registers <a id="addition1"></a>

In this previous algorithm, we had to pass to the operator the value of $k$ that we wanted to add to the starting state. But at other times, instead of passing that value to the operator, we would like it to pull the information from another register. That is, we are looking for a new operator $\text{Sum}_2$ such that


$$\text{Sum}_2 |m\rangle |k\rangle |0\rangle = |m\rangle |k\rangle |m+k\rangle$$

In this case, we can understand the third register (which is initially at $0$) as a counter that will tally as many units as $m$ and $k$ combined. The binary decomposition will make this simple. If we have $|m\rangle = |q_0q_1q_2\rangle$, we will have to add $1$ to the counter if $q_2 = 1$ and nothing otherwise. In general, we should add $2^{n-i-1}$ units if the $i$-th qubit is in state $|1\rangle$ and $0$ otherwise. As we can see, this is the same idea that is also behind the concept of a controlled gate. Indeed, observe that we will, indeed, apply a corresponding phase if indeed the control qubit is in the state $|1\rangle$. Let us now code the $\text{Sum}_2$ operator.


In [5]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer, AerSimulator
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_histogram
import numpy as np

# we use the little endian notation for bit ordering

# Define a function for the controlled addition in the Fourier basis
def add_k_fourier(circuit, control, k, wires):
    for j in range(len(wires)):
        circuit.crz(k * np.pi/(2**(len(wires) - 1 - j)), control, wires[j])

# Define the basis_embedding function
def basis_embedding(circuit, m, wires):
    binary_m = bin(m)[2:].zfill(len(wires))
    print(binary_m)
    for i, bit in enumerate(binary_m):
        if bit == '1':
            circuit.x(wires[-1-i])

def addition(circuit, wires_m, wires_k, wires_solution):
    # Apply the QFT to the solution qubits
    circuit.append(QFT(len(wires_solution)), wires_solution)

    # Add m to the counter
    for i in range(len(wires_m)):
        add_k_fourier(circuit, wires_m[i], 2**i, wires_solution)

    # Add k to the counter
    for i in range(len(wires_k)):
        add_k_fourier(circuit, wires_k[i], 2**i, wires_solution)

    # Apply the inverse QFT to the solution qubits
    circuit.append(QFT(len(wires_solution)).inverse(), wires_solution)

def sum2(m, k, wires_m, wires_k, wires_solution):
    # Define the total number of qubits
    n_wires = max(wires_solution) + 1

    # Create a quantum circuit
    circuit = QuantumCircuit(n_wires, len(wires_solution))

    # m and k codification
    basis_embedding(circuit, m, wires_m)
    basis_embedding(circuit, k, wires_k)

    # apply the addition circuit
    addition(circuit, wires_m, wires_k, wires_solution)

    # Measure the solution qubits
    circuit.measure(wires_solution, list(range(len(wires_solution))))

    print(circuit.draw())

    # Execute the circuit
    sim = Aer.get_backend('aer_simulator')
    circuit = transpile(circuit, AerSimulator())
    result = sim.run(circuit).result()
    counts = result.get_counts()

    # Print the result
    counts = result.get_counts(circuit)
    print(f"The result of the sum is {counts}")

    # Plot the result
    plot_histogram(counts)

# Define m and k
m = 7
k = 6

# Define the number of qubits for m, k, and the solution
wires_m = [0, 1, 2]
wires_k = [3, 4, 5]
wires_solution = [6, 7, 8, 9]

sum2(m, k, wires_m, wires_k, wires_solution)

/var/folders/0y/p40nt1td47xgs3zh4yrj4qwc0000gn/T/ipykernel_24294/854869615.py:24: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  circuit.append(QFT(len(wires_solution)), wires_solution)
/var/folders/0y/p40nt1td47xgs3zh4yrj4qwc0000gn/T/ipykernel_24294/854869615.py:35: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  circuit.append(QFT(len(wires_solution)).inverse(), wires_solution)


111
110
      ┌───┐                                                                    »
q_0: ─┤ X ├───────■──────────■─────────────────────■───────────────────────────»
      ├───┤       │          │                     │                           »
q_1: ─┤ X ├───────┼──────────┼──────────■──────────┼──────────■────────────────»
      ├───┤       │          │          │          │          │                »
q_2: ─┤ X ├───────┼──────────┼──────────┼──────────┼──────────┼──────────■─────»
      └───┘       │          │          │          │          │          │     »
q_3: ─────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────»
      ┌───┐       │          │          │          │          │          │     »
q_4: ─┤ X ├───────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────»
      ├───┤       │          │          │          │          │          │     »
q_5: ─┤ X ├───────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────»
     ┌┴───┴─┐┌────┴─